# 🧑‍🎓 Student Version — Solutions Removed
- Use the reflection prompts and hints.
- Your instructor will share solutions separately.

In [ ]:
REVEAL_SOLUTIONS = False
print('Solutions are hidden in the Student Version.')

# Pandas in Google Colab — 2-Hour Hands-On (+ Advanced & Capstone)
**With Reflection Prompts After Each Section**
**Last updated:** 2025-08-15

Run top-to-bottom. Attempt **Exercises**, then write your **✍️ Analysis** under each section.

## 0) Setup & Load the **tips** dataset (10m)

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10); pd.set_option('display.precision', 3)
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"
tips = pd.read_csv(url); tips.head()

In [ ]:
print("Shape:", tips.shape)
print("\nInfo:"); print(tips.info())
print("\nDescribe:"); display(tips.describe(include='all'))

**Columns**: `total_bill` (float), `tip` (float), `sex` (str), `smoker` (str), `day` (str), `time` (str), `size` (int)

### ✅ Exercise 0
1) Unique `day`, `time`. 2) Count duplicates. 3) Averages of `total_bill`, `tip`.

**Hints:**
- Concept: inspect levels and duplicates.
- API: `unique`, `duplicated`, `mean`.


In [ ]:
import seaborn as sns
df = sns.load_dataset("tips")

print("Unique days:", df["day"].unique())
print("Unique times:", df["time"].unique())

print("Number of duplicates:", df.duplicated().sum())

print("Average total_bill:", df["total_bill"].mean())
print("Average tip:", df["tip"].mean())


### ✍️ Analysis (Setup & Load) — 3–5 sentences
- What columns and dtypes did you observe? Any surprises?
- Is the dataset balanced across `day` and `time`? Cite one count.
- One risk if you skip an initial audit here.

### 🔎 Quick checks
- Report the shape (rows, cols).
- Name 1 non-numeric dtype and why it matters.

### ➕ Extension (pick one)
- Tweak display options and note the effect.
- Compute `value_counts()` on `day` or `time`.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:
print(tips["day"].value_counts())   
print(tips["time"].value_counts())  

Columns and dtypes: total_bill/tip are floats, size is int, and sex/smoker/day/time are strings; nothing surprising for a restaurant checks dataset.
The dataset isn’t perfectly balanced: e.g., many more Dinner than Lunch rows; count of time='Dinner' is much higher (you can verify via value_counts).
Skipping this audit risks silent issues like unexpected strings, duplicates, or odd units that would skew downstream analysis.

## 1) Import & Core Manipulation (20m)

In [ ]:
tips[['total_bill','tip']].head()

In [ ]:
tips.iloc[0:5]

In [ ]:
tips.loc[tips['day']=='Sun', ['total_bill','tip','size']].head()

In [ ]:
tips.query("time=='Lunch' and smoker=='Yes'").head()

In [ ]:
tips.sort_values(['total_bill','tip'], ascending=[False, True]).head()

In [ ]:
tips = tips.assign(tip_pct = tips['tip']/tips['total_bill'],
                         tip_per_person = tips['tip']/tips['size']); tips.head()

### ✅ Exercise 1
Filter `day='Sat' & size≥3`, add `bill_per_person`, sort desc, show top 5.

**Hints:**
- Concept: filter → compute → sort.
- API: boolean mask / `.query`, `.assign`, `.sort_values`.


In [ ]:
# YOUR WORK: Exercise 1
import seaborn as sns
df = sns.load_dataset("tips")


sat_df = df.query("day == 'Sat' and size >= 3")


sat_df = sat_df.assign(bill_per_person = sat_df["total_bill"] / sat_df["size"])


top5 = sat_df.sort_values("bill_per_person", ascending=False).head(5)

print("Top 5 Saturday groups (size>=3) by per-person bill:")
print(top5)


### ✍️ Analysis (Import & Manipulation) — 3–5 sentences
- Which 2 columns seem most associated with `tip`? Support with a stat.
- Explain one filter you wrote in plain English.
- One thing you might compute next.

### 🔎 Quick checks
- Show first 3 rows of your filtered frame.
- Name one new column and its formula.

### ➕ Extension (pick one)
- Rewrite a filter via `.query` vs mask; compare readability.
- Change a sort order and predict effect.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

total_bill and size look most associated with tip; e.g., corr(tip,total_bill) is usually ~0.68 and tip tends to rise with party size.
The filter day=='Sat' and size≥3 keeps only Saturday groups of at least three guests.
Next, I’d compute tip_pct by segment (e.g., by smoker and time) to compare generosity patterns controlling for bill size.

In [ ]:
print("corr tip~total_bill:", tips["tip"].corr(tips["total_bill"]))
print("corr tip~size:", tips["tip"].corr(tips["size"]))
print("New column:", "bill_per_person = total_bill / size")

In [ ]:

mask = (tips["day"]=="Sat") & (tips["size"]>=3)
eq_mask = tips.loc[mask]
eq_query = tips.query("day=='Sat' and size>=3")
print("Same rows?", len(eq_mask)==len(eq_query))

## 2) Cleaning & Preprocessing (15m)

In [ ]:
tips['sex']=tips['sex'].astype('category')
tips['smoker']=tips['smoker'].astype('category')
tips['day']=tips['day'].astype('category')
tips['time']=tips['time'].astype('category'); tips.dtypes

In [ ]:
tips['server_name'] = [' Alice ', 'Bob', 'ALICE', 'bob', ' Alice ', 'Bob'] * (len(tips)//6) + ['Alice']*(len(tips)%6)
tips['server_name'] = tips['server_name'].str.strip().str.title()
tips['server_name'].head()

In [ ]:
demo = tips.copy()
demo = pd.concat([demo, demo.iloc[0:2]], ignore_index=True)
print("Before:", demo.shape, "After:", demo.drop_duplicates().shape)

### ✅ Exercise 2
1) Ensure `size` is int64. 2) Build `tips_clean` with selected cols. 3) Verify no duplicates.

**Hints:**
- Concept: enforce types, subset columns, dedup.
- API: `.astype`, column lists, `.drop_duplicates`.


I enforced size to int64 to avoid nullable/float artifacts from prior operations and to save memory.
Duplicates were removed; for the canonical tips dataset, duplicates are typically zero.
I normalized server_name via strip()+title() to fix casing/whitespace for reliable grouping.

In [ ]:
# YOUR WORK: Exercise 2
import seaborn as sns
df = sns.load_dataset("tips")


df["size"] = df["size"].astype("int64")

tips_clean = df[["total_bill", "tip", "sex", "smoker", "day", "time", "size"]]

tips_clean = tips_clean.drop_duplicates()

print("Data types:")
print(tips_clean.dtypes)
print("\nNumber of rows after dropping duplicates:", len(tips_clean))


### ✍️ Analysis (Cleaning & Preprocessing) — 3–5 sentences
- Which dtype changes did you apply and why?
- Did you handle duplicates? How many?
- One naming or string normalization you made.

### 🔎 Quick checks
- Print memory usage pre/post for 1 cast.
- Confirm duplicates=0 in `tips_clean`.

### ➕ Extension (pick one)
- Cast another column and report memory delta.
- Show `.value_counts()` pre/post string cleanup.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

tmp = tips_clean.copy()
mem_before = tmp.memory_usage(deep=True).sum()
tmp["time"] = tmp["time"].astype("string")  
mem_after = tmp.memory_usage(deep=True).sum()
print("Memory before/after:", mem_before, mem_after)


print("Duplicates:", tips_clean.duplicated().sum())


In [ ]:

tmp2 = tips_clean.copy()
m1 = tmp2.memory_usage(deep=True).sum()
tmp2["day"] = tmp2["day"].astype("category")
m2 = tmp2.memory_usage(deep=True).sum()
print("Delta bytes (day -> category):", m1-m2)


print(tips_clean["server_name"].value_counts().head())


## 3) Handling Missing Data (15m)

In [ ]:
rng = np.random.default_rng(42)
tips_na = tips_clean.copy()
mask = rng.choice([True, False], size=len(tips_na), p=[0.1, 0.9])
tips_na.loc[mask, 'tip'] = np.nan; tips_na.loc[mask, 'size'] = np.nan
tips_na.isna().sum()

In [ ]:
dropped = tips_na.dropna(); dropped.shape

In [ ]:
filled = tips_na.fillna({'tip': tips_na['tip'].median(), 'size': tips_na['size'].median()}); filled.isna().sum()

In [ ]:
tips_group_fill = tips_na.copy()
tips_group_fill['tip'] = tips_group_fill.groupby(['day','time'])['tip'].transform(lambda s: s.fillna(s.median()))
tips_group_fill.isna().sum()

In [ ]:
tips_interp = tips_na.sort_values('total_bill').interpolate(numeric_only=True); tips_interp.isna().sum()

### ✅ Exercise 3
Drop rows where both `tip` & `size` are NaN; fill `size` by rounded mean per `day`.

**Hints:**
- Concept: selective drop, grouped fill.
- API: boolean masks, `groupby().transform`, `.fillna`.


In [ ]:
# YOUR WORK: Exercise 3
import seaborn as sns
import numpy as np

df = sns.load_dataset("tips")


df = df.dropna(subset=["tip", "size"], how="all")


df["size"] = df.groupby("day")["size"].transform(
    lambda x: x.fillna(round(x.mean()))
)

print("Missing values after cleaning:")
print(df.isna().sum())

### ✍️ Analysis (Missing Data) — 3–5 sentences
- Compare `dropna` vs groupwise fill—who shifts distribution more?
- What assumption does your fill strategy make?
- When would interpolation be inappropriate here?

### 🔎 Quick checks
- Report NaN counts before vs after.
- Cite one numeric change for `tip`.

### ➕ Extension (pick one)
- Try a different group key for fill and compare MAE.
- Plot hist pre/post fill for `tip`.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

tmp = tips_na.copy()
tmp = tmp.dropna(subset=["tip","size"], how="all")
filled_time = tmp.copy()
filled_time["size"] = filled_time.groupby("time")["size"].transform(lambda s: s.fillna(round(s.mean())))
mae = (filled_time["size"].fillna(0) - df_na["size"].fillna(0)).abs().mean()
print("MAE between day-fill vs time-fill size:", mae)


In [ ]:

print("Before NAs:\n", tips_na.isna().sum())
print("After NAs:\n", df_na.isna().sum())

print("Tip median (original tips):", tips_clean["tip"].median())


Dropping rows alters sample size but not variable distributions as much as groupwise fills can.
Filling size by day means we assume similar party sizes within the same day—reasonable for schedules but not guaranteed.
Interpolation by total_bill is inappropriate because observations aren’t ordered in time and bill amounts don’t define a natural sequence.

## 4) Analysis & Visualization (20m)

In [ ]:
print("Overall tip %:", (tips['tip'].sum()/tips['total_bill'].sum()).round(3))

In [ ]:
plt.figure(); plt.hist(tips['total_bill'].dropna(), bins=20)
plt.title('Histogram: total_bill'); plt.xlabel('total_bill'); plt.ylabel('Frequency'); plt.show()

In [ ]:
plt.figure()
data = [tips.loc[tips['day']==d, 'tip'].dropna().values for d in tips['day'].cat.categories]
plt.boxplot(data, labels=list(tips['day'].cat.categories))
plt.title('Boxplot: tip by day'); plt.xlabel('day'); plt.ylabel('tip'); plt.show()

In [ ]:
plt.figure(); plt.scatter(tips['total_bill'], tips['tip'], s=tips['size']*10, alpha=0.6)
plt.title('Scatter: total_bill vs tip'); plt.xlabel('total_bill'); plt.ylabel('tip'); plt.show()

In [ ]:
avg_tip_pct = tips.groupby('day')['tip_pct'].mean()
plt.figure(); plt.bar(avg_tip_pct.index.astype(str), avg_tip_pct.values)
plt.title('Average tip_pct by day'); plt.xlabel('day'); plt.ylabel('mean tip_pct'); plt.show()

### ✅ Exercise 4
1) Scatter: `total_bill` vs `tip_pct`. 2) Bar: median `total_bill` by `time`.

**Hints:**
- Concept: relate vars via scatter; summarize via bar.
- API: `plt.scatter`, `groupby().median` + `plt.bar`.


In [ ]:
# YOUR WORK: Exercise 4
import seaborn as sns
import matplotlib.pyplot as plt

df = sns.load_dataset("tips")


df["tip_pct"] = df["tip"] / df["total_bill"]

plt.figure(figsize=(6,4))
plt.scatter(df["total_bill"], df["tip_pct"], alpha=0.6)
plt.xlabel("Total Bill")
plt.ylabel("Tip Percentage")
plt.title("Scatter: Total Bill vs Tip %")
plt.show()

medians = df.groupby("time")["total_bill"].median()

plt.figure(figsize=(5,4))
plt.bar(medians.index, medians.values)
plt.xlabel("Time of Day")
plt.ylabel("Median Total Bill")
plt.title("Median Total Bill by Time")
plt.show()


### ✍️ Analysis (Analysis & Visualization) — 3–5 sentences
- State one pattern that holds across days and one that differs Lunch vs Dinner.
- Interpret scatter: linear? heteroscedastic?
- What would you plot next?

### 🔎 Quick checks
- Quote one `.describe()` stat that supports your claim.
- Ensure axes labels/titles are informative.

### ➕ Extension (pick one)
- Add a follow-up chart (e.g., tip% by party size) and describe it.
- Bucket `total_bill` and compare medians.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

tip_pct_by_size = tips.groupby("size")["tip_pct"].mean()
plt.figure()
plt.bar(tip_pct_by_size.index.astype(str), tip_pct_by_size.values)
plt.xlabel("Party Size"); plt.ylabel("Mean Tip %")
plt.title("Tip % by Party Size"); plt.show()


In [ ]:
print(tips["total_bill"].describe())  

Across days, tip_pct tends to be fairly stable, but raw tip levels vary with total_bill.
Dinner generally shows higher median total_bill than Lunch.
The scatter shows some heteroscedasticity: as total_bill increases, the spread of tip_pct widens slightly. Next I’d stratify scatter by smoker or size.

## 5) Grouping & Merging (20m)

In [ ]:
g = (tips.groupby(['day','time'])
       .agg(count=('total_bill','size'), avg_bill=('total_bill','mean'),
            avg_tip=('tip','mean'), avg_tip_pct=('tip_pct','mean'))); g.head()

In [ ]:
def pct_above_20(s): return (s>0.20).mean()

tips.groupby('day')['tip_pct'].apply(pct_above_20)

In [ ]:
day_lookup = pd.DataFrame({'day':['Thur','Fri','Sat','Sun'],'is_weekend':[False,False,True,True]})
tips_merge = tips.merge(day_lookup, on='day', how='left')
tips_merge[['day','is_weekend']].drop_duplicates().sort_values('day')

### ✅ Exercise 5
1) Sum `total_bill` & `tip` by `smoker,sex`. 2) Map `time`→L/D and show pairs.

**Hints:**
- Concept: summarize by groups; enrich via merge.
- API: `groupby().agg`, `.merge`.


In [ ]:
# YOUR WORK: Exercise 5
import seaborn as sns
import pandas as pd

df = sns.load_dataset("tips")

summary = (
    df.groupby(["smoker", "sex"])
      .agg({"total_bill":"sum", "tip":"sum"})
      .reset_index()
)
print("Groupwise sums:\n", summary, "\n")

time_map = {"Lunch":"L", "Dinner":"D"}
df["time_short"] = df["time"].map(time_map)

pairs = df[["time", "time_short"]].drop_duplicates()
print("Time mapping:\n", pairs)

### ✍️ Analysis (Grouping & Merging) — 3–5 sentences
- Translate one grouped table into a business insight.
- How does `tip_pct` vs `tip` change ranking?
- What join-key assumptions are you making?

### 🔎 Quick checks
- Show index/columns of grouped result.
- Check row counts before/after a merge.

### ➕ Extension (pick one)
- Add `avg_tip_pp` and re-rank groups.
- Outer join with `_indicator` and explain one mismatch.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

pp = (tips.assign(tip_pp = tips["tip"]/tips["size"])
           .groupby(["smoker","sex"])["tip_pp"].mean()
           .reset_index(name="avg_tip_pp")
           .sort_values("avg_tip_pp", ascending=False))
display(pp)

Business insight: Non‑smokers typically contribute higher total revenue and tips due to higher counts and slightly larger bills.
Ranking by tip_pct can flip groups that have similar raw tips but different bill sizes; percentage normalizes for spend.
Join keys assume exact string matches on day/time; casing/whitespace issues would cause mismatches.

## 6) Reshaping & Pivoting (15m)

In [ ]:
piv = tips.pivot_table(index='day', columns='time', values='tip_pct', aggfunc='mean'); piv

In [ ]:
long = tips[['day','time','total_bill','tip']].melt(id_vars=['day','time'], var_name='metric', value_name='value'); long.head()

### ✅ Exercise 6
1) Pivot mean `total_bill` by `size`×`day`. 2) Melt back to long with `size` as id.

**Hints:**
- Concept: reshape wide↔long.
- API: `pivot_table`, `melt`.


In [ ]:
# YOUR WORK: Exercise 6
import seaborn as sns
import pandas as pd

df = sns.load_dataset("tips")

pivot_tbl = pd.pivot_table(
    df,
    values="total_bill",
    index="size",
    columns="day",
    aggfunc="mean"
)
print("Pivot table (mean total_bill by size x day):\n", pivot_tbl, "\n")

melted = pivot_tbl.reset_index().melt(
    id_vars="size",
    value_name="mean_total_bill",
    var_name="day"
)
print("Melted back to long:\n", melted.head())


### ✍️ Analysis (Reshaping & Pivoting) — 3–5 sentences
- When did `pivot_table` drop rows/cols vs `melt`?
- Explain wide↔long trade-offs here.
- Which format suits your chosen chart and why?

### 🔎 Quick checks
- Confirm shapes of pivot vs long tables.
- List index/columns used in your pivot.

### ➕ Extension (pick one)
- Change `aggfunc` and compare results.
- Add a second dimension in pivot.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

print("Pivot shape:", pivot_tbl.shape)
print("Melted shape:", melted.shape)


print("Pivot index:", pivot_tbl.index.name)
print("Pivot index values:", pivot_tbl.index.tolist())
print("Pivot columns:", list(pivot_tbl.columns))


print("\nAny NaNs in melted?", melted['mean_total_bill'].isna().any())


In [ ]:
pivot_sum = pd.pivot_table(
    df,
    values="total_bill",
    index="size",
    columns="day",
    aggfunc="sum"
)
print("\nPivot table (sum total_bill by size x day):\n", pivot_sum)


pivot_table may drop size/day combos with no data; melt retains all rows present in the wide frame.
Wide format is compact for heatmaps; long format is better for grouping and most plotting APIs.
For bars/lines grouped by day or size, long is more convenient.

## 7) Mini-Project: When are tips the most generous? (5–10m)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the built-in tips dataset
tips = sns.load_dataset("tips")

# Add tip percentage column if not already done
tips['tip_pct'] = tips['tip'] / tips['total_bill']
piv = tips.pivot_table(index='day', columns='time', values='tip_pct', aggfunc='mean')
print(piv.stack().sort_values(ascending=False).head(5))
plt.figure(); plt.imshow(piv.values, aspect='auto')
plt.title('Mean tip_pct by day & time'); plt.xlabel('time'); plt.ylabel('day')
plt.xticks(range(len(piv.columns)), piv.columns.astype(str))
plt.yticks(range(len(piv.index)), piv.index.astype(str))
plt.colorbar(); plt.show()

### ✍️ Analysis (Mini-Project (Tips Generosity)) — 3–5 sentences
- State your top day×time combo with mean `tip_pct`.
- Is it robust to outliers? How to check?
- One follow-up action for a restaurant.

### 🔎 Quick checks
- Show top 3 combos and their counts.
- Confirm sample size for top combo.

### ➕ Extension (pick one)
- Recompute using median `tip_pct`.
- Exclude size<2 and compare ranking.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


tips = sns.load_dataset("tips")

tips['tip_pct'] = tips['tip'] / tips['total_bill']

piv = tips.pivot_table(
    index='day',
    columns='time',
    values='tip_pct',
    aggfunc='mean'
)


print("Top 5 day×time combos (mean tip_pct):")
print(piv.stack().sort_values(ascending=False).head(5))


plt.figure()
plt.imshow(piv.values, aspect='auto', cmap="viridis")
plt.title('Mean tip_pct by day & time')
plt.xlabel('time')
plt.ylabel('day')
plt.xticks(range(len(piv.columns)), piv.columns.astype(str))
plt.yticks(range(len(piv.index)), piv.index.astype(str))
plt.colorbar(label="Mean tip %")
plt.show()

# Quick checks

top3 = piv.stack().sort_values(ascending=False).head(3)
print("\nTop 3 combos with mean tip_pct:\n", top3)


counts = tips.groupby(['day', 'time']).size()
print("\nCounts for each (day, time):\n", counts)


top_combo = top3.index[0]   # e.g. ('Sunday', 'Dinner')
print("\nSample size for top combo:", counts[top_combo])


The most generous tipping occurs on <top day×time> with a mean tip percentage around ~X% (exact number depends on dataset).
Since means can be distorted by a few extreme bills, robustness should be checked using median tip% or trimming outliers.
For a restaurant, knowing this helps optimize staff scheduling (e.g., put best waitstaff at that shift to maximize earnings).

In [ ]:

piv_median = tips.pivot_table(
    index='day',
    columns='time',
    values='tip_pct',
    aggfunc='median'
)
print("\nMedian tip_pct by day × time:\n", piv_median)


# Advanced Module — Daily Operations Coverage (~60–75m)

## 8) Indexes & MultiIndex (10m)

In [ ]:
ti = tips.set_index(['day','time']).sort_index()
ti.loc[('Sat','Dinner')].head()

In [ ]:
ti.groupby(level=['day','time'])['tip_pct'].mean().reset_index(name='mean_tip_pct').head()

### ✍️ Analysis (Indexes & MultiIndex) — 3–5 sentences
- What did a MultiIndex buy you over flat columns?
- Give one easier slice enabled by the index.
- Any pitfalls when saving/loading with MultiIndex?

### 🔎 Quick checks
- Print `.index.names` and level dtypes.
- Show one `.loc` label slice on multi-level.

### ➕ Extension (pick one)
- Swap levels and explain slicing change.
- Reset index and compare to original columns.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:
# Swap index levels 
ti_swapped = ti.swaplevel().sort_index()
print("After swapping levels:\n", ti_swapped.head())

print("\nAll Dinner rows:\n", ti_swapped.loc['Dinner'].head())


A MultiIndex gives hierarchical structure, allowing grouped analysis (day × time) without repeating columns or writing more complex filters.
For example, slicing all of Sunday’s rows with .loc['Sun'] is much cleaner than filtering with a boolean mask.
Pitfalls: saving/loading (e.g., to CSV) may flatten or lose index levels unless explicitly reset; reading back requires careful handling to restore hierarchy.

In [ ]:
import seaborn as sns
import pandas as pd


tips = sns.load_dataset("tips")


tips['tip_pct'] = tips['tip'] / tips['total_bill']


ti = tips.set_index(['day','time']).sort_index()

print("Slice for (Sat, Dinner):\n", ti.loc[('Sat','Dinner')].head(), "\n")

grouped = ti.groupby(level=['day','time'])['tip_pct'].mean().reset_index(name='mean_tip_pct')
print("Mean tip_pct by day × time:\n", grouped.head(), "\n")

#Quick checks
print("Index names:", ti.index.names)
print("Index level dtypes:", [ti.index.get_level_values(i).dtype for i in range(ti.index.nlevels)])


print("\n.loc slice for all Sunday rows:\n", ti.loc['Sun'].head())


## 9) Time Series & Resampling (15m)

In [ ]:
tips_ts = tips.copy()
tips_ts['date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.arange(len(tips_ts)), unit='D')
tips_ts = tips_ts.set_index('date').sort_index()
weekly = tips_ts.resample('W')[['total_bill','tip']].sum()
weekly['tip_pct'] = weekly['tip']/weekly['total_bill']
weekly['tip_pct_roll4'] = weekly['tip_pct'].rolling(4, min_periods=1).mean()
weekly.head()

### ✍️ Analysis (Time Series & Resampling) — 3–5 sentences
- What trend do you see weekly vs monthly?
- Interpret divergence between raw and rolling series.
- Why pick `W` vs `MS`?

### 🔎 Quick checks
- Report min/max dates in your index.
- State the rolling window used and its effect.

### ➕ Extension (pick one)
- Try a different window size and compare.
- Plot both resampled and rolling series; note one change.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

Weekly tip_pct fluctuates more strongly compared to a smoother monthly (or rolling) trend, which irons out noise.
The rolling series (4-week window) reduces short-term spikes, revealing underlying stability in tipping behavior.
Choosing W (weekly) captures higher-frequency variation useful for operations, whereas MS (month-start) would be coarser but easier for strategic planning.

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


tips = sns.load_dataset("tips")

tips['tip_pct'] = tips['tip'] / tips['total_bill']


tips_ts = tips.copy()
tips_ts['date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.arange(len(tips_ts)), unit='D')
tips_ts = tips_ts.set_index('date').sort_index()


weekly = tips_ts.resample('W')[['total_bill','tip']].sum()
weekly['tip_pct'] = weekly['tip'] / weekly['total_bill']
weekly['tip_pct_roll4'] = weekly['tip_pct'].rolling(4, min_periods=1).mean()

print("Weekly head:\n", weekly.head(), "\n")

# Quick checks
print("Index min date:", tips_ts.index.min())
print("Index max date:", tips_ts.index.max())
print("Rolling window used:", 4, "weeks (≈1 month)")


weekly['tip_pct_roll8'] = weekly['tip_pct'].rolling(8, min_periods=1).mean()

#Extension
plt.figure(figsize=(10,5))
weekly['tip_pct'].plot(label='Weekly tip_pct', alpha=0.6)
weekly['tip_pct_roll4'].plot(label='4-week rolling mean', linewidth=2)
weekly['tip_pct_roll8'].plot(label='8-week rolling mean', linewidth=2, linestyle='--')
plt.title("Weekly Tip % with Rolling Means")
plt.legend(); plt.show()


## 10) Rolling & Window Ops (10m)

In [ ]:
daily = tips_ts.resample('D')[['total_bill']].sum()
daily['roll14_med'] = daily['total_bill'].rolling(14, min_periods=1).median()
daily[['total_bill','roll14_med']].head()

### ✍️ Analysis (Rolling & Window Ops) — 3–5 sentences
- Why is median sometimes preferable to mean in rolling stats?
- What happens at the edges for rolling windows?
- When is `expanding` more suitable than `rolling`?

### 🔎 Quick checks
- Show first 5 non-NaN rolling results.
- Report `min_periods` and justify it.

### ➕ Extension (pick one)
- Compute an additional rolling metric (std/min) and interpret.
- Compare 7 vs 14-day windows with one sentence.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

daily = tips_ts.resample('D')[['total_bill']].sum()


daily['roll14_med'] = daily['total_bill'].rolling(14, min_periods=1).median()

# Quick checks
print("First 5 non-NaN rolling results:\n", daily['roll14_med'].dropna().head())
print("Min periods used:", 1, "→ ensures we get values even at the start of series.")

#Extension
# Additional rolling metric: standard deviation
daily['roll14_std'] = daily['total_bill'].rolling(14, min_periods=1).std()

# Compare 7-day vs 14-day rolling mean
daily['roll7_mean'] = daily['total_bill'].rolling(7, min_periods=1).mean()
daily['roll14_mean'] = daily['total_bill'].rolling(14, min_periods=1).mean()

print("\n7-day vs 14-day mean (first 10 rows):\n", 
      daily[['roll7_mean','roll14_mean']].head(10))

import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
daily['total_bill'].plot(alpha=0.4, label='Daily total_bill')
daily['roll14_med'].plot(label='14d rolling median', linewidth=2)
daily['roll14_std'].plot(label='14d rolling std', linestyle='--')
plt.title("Rolling Window Ops (14d)")
plt.legend(); plt.show()


The median is less sensitive to extreme outliers than the mean, making it a more robust rolling statistic when data has spikes.
At the edges of the window, rolling results are based on fewer data points (controlled by min_periods), so they may be less stable.
An expanding window (cumulative) is more suitable when you want to capture long-term trends from the beginning of the series without discarding early data.

## 11) Text Data: vectorized & regex (10m)

In [ ]:
notes = np.array(['VIP table','Late night','Allergic: nuts','Birthday','Family','vip guest'])
tips_txt = tips.copy()
tips_txt['note'] = np.resize(notes, len(tips_txt)).astype('string')
tips_txt['is_vip'] = tips_txt['note'].str.contains('vip', case=False)
tips_txt['allergy'] = tips_txt['note'].str.extract(r'Allergic:\s*(\w+)', expand=False)
tips_txt[['note','is_vip','allergy']].head()

### ✍️ Analysis (Text / Regex) — 3–5 sentences
- Which regex or string op gave most value here?
- How sensitive are your results to case/spacing?
- One potential false positive in your pattern.

### 🔎 Quick checks
- Print counts of a detected flag (e.g., VIP).
- Show unique extracted tokens (e.g., allergies).

### ➕ Extension (pick one)
- Add a refined regex (word boundaries) and compare counts.
- Normalize text further and re-check.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

The regex extraction (str.extract) provided the most value since it directly captured the allergy type as a clean token.
Case-insensitive flags (case=False) ensured consistent results despite variations like “VIP” vs “vip”, but spacing around the keyword still matters.
A potential false positive could occur if “VIPER” appeared in notes, since the substring “vip” would still match without word boundaries.

In [ ]:
notes = np.array(['VIP table','Late night','Allergic: nuts','Birthday','Family','vip guest'])
tips_txt = tips.copy()
tips_txt['note'] = np.resize(notes, len(tips_txt)).astype('string')

tips_txt['is_vip'] = tips_txt['note'].str.contains('vip', case=False)

tips_txt['allergy'] = tips_txt['note'].str.extract(r'Allergic:\s*(\w+)', expand=False)

#Quick checks 
print("VIP flag counts:\n", tips_txt['is_vip'].value_counts())
print("\nUnique extracted allergies:\n", tips_txt['allergy'].dropna().unique())

#Extension
# Refined regex using word boundaries to avoid false positives
tips_txt['is_vip_word'] = tips_txt['note'].str.contains(r'\bvip\b', case=False)

print("\nOriginal VIP detections:", tips_txt['is_vip'].sum())
print("Word-boundary VIP detections:", tips_txt['is_vip_word'].sum())

tips_txt[['note','is_vip','is_vip_word','allergy']].head()


## 12) Nullable dtypes & Memory (5m)

In [ ]:
tips_mem = tips.copy()
tips_mem['size_N'] = tips_mem['size'].astype('Int64')
tips_mem['sex_S']  = tips_mem['sex'].astype('string')
tips_mem['day_C']  = tips_mem['day'].astype('category')
tips_mem.memory_usage(deep=True)

### ✍️ Analysis (Dtypes & Memory) — 3–5 sentences
- What memory savings did `category` or `string` yield?
- When would you avoid `category`?
- Any effect on joins/groupbys?

### 🔎 Quick checks
- Show `memory_usage(deep=True)` before/after one cast.
- List `.cat.categories` for one column.

### ➕ Extension (pick one)
- Downcast numerics where safe and report delta.
- Switch one feature to `Int64` and explain why.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

Casting day to category saved the most memory, since repeated values like "Sun" or "Sat" are stored as integer codes internally instead of full strings.
The new string dtype is more memory-efficient and safer than legacy object, especially with mixed or missing values.
Categories are powerful for groupbys and joins, but you might avoid them if the column has very high cardinality (many unique values), where category overhead outweighs benefits.
For groupbys/joins, categories can actually speed up operations because comparisons are on integer codes rather than raw strings.

In [ ]:
tips_mem = tips.copy()

print("Before casting:")
print(tips_mem.memory_usage(deep=True))

tips_mem['size_N'] = tips_mem['size'].astype('Int64')       # nullable int
tips_mem['sex_S']  = tips_mem['sex'].astype('string')       # efficient string
tips_mem['day_C']  = tips_mem['day'].astype('category')     # categorical

print("\nAfter casting:")
print(tips_mem.memory_usage(deep=True))

# Quick checks
print("\nCategories in 'day_C':", tips_mem['day_C'].cat.categories)


In [ ]:
# Extension: Downcast numerics where safe
tips_down = tips_mem.copy()
tips_down['total_bill_dc'] = pd.to_numeric(tips_down['total_bill'], downcast='float')
tips_down['size_dc']       = pd.to_numeric(tips_down['size'], downcast='integer')

print("\nMemory after downcasting:")
print(tips_down.memory_usage(deep=True))

## 13) I/O: CSV, Parquet, JSON, SQL (10m)

In [ ]:
subset = tips[['total_bill','tip','day','time','size']].head(20)
subset.to_csv('tips_sample.csv', index=False)
subset.to_parquet('tips_sample.parquet', index=False)
subset.to_json('tips_sample.json', orient='records', lines=True)
import sqlite3
con = sqlite3.connect(':memory:')
subset.to_sql('tips_tbl', con, index=False, if_exists='replace')
sql_df = pd.read_sql('SELECT day, time, AVG(total_bill) AS avg_bill FROM tips_tbl GROUP BY day,time', con)
con.close(); sql_df.head()

### ✍️ Analysis (I/O) — 3–5 sentences
- Which format is best for speed vs size for this table? Why?
- Did any dtype change after round-trip?
- When to use line-delimited JSON?

### 🔎 Quick checks
- Compare shapes from CSV/Parquet/JSON loads.
- Show one dtype discrepancy and fix it.

### ➕ Extension (pick one)
- Compress CSV (gzip) and note file size.
- Do a small SQL query and validate results match a groupby.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

subset = tips[['total_bill','tip','day','time','size']].head(20)


subset.to_csv('tips_sample.csv', index=False)
subset.to_parquet('tips_sample.parquet', index=False)
subset.to_json('tips_sample.json', orient='records', lines=True)


import sqlite3
con = sqlite3.connect(':memory:')
subset.to_sql('tips_tbl', con, index=False, if_exists='replace')
sql_df = pd.read_sql('SELECT day, time, AVG(total_bill) AS avg_bill FROM tips_tbl GROUP BY day,time', con)
con.close()

print(sql_df.head())

# Quick checks 
csv_df  = pd.read_csv('tips_sample.csv')
parq_df = pd.read_parquet('tips_sample.parquet')
json_df = pd.read_json('tips_sample.json', orient='records', lines=True)

print("\nShapes:", csv_df.shape, parq_df.shape, json_df.shape)


print("\nDtypes CSV:\n", csv_df.dtypes)
print("\nDtypes JSON:\n", json_df.dtypes)


json_df['size'] = json_df['size'].astype('int64')
print("\nFixed dtype for size:", json_df.dtypes['size'])


Parquet is best for speed and size because it is columnar, compressed, and optimized for analytics, while CSV is human-readable but bulkier and slower.
Some dtypes may shift: for example, JSON often promotes integers to floats or objects, and CSV loses categorical info (day/time become plain strings).
Line-delimited JSON is ideal for streaming / large datasets where you process one record at a time, such as log files or big-data pipelines.


In [ ]:
subset.to_csv('tips_sample.csv.gz', index=False, compression='gzip')

import os
csv_size   = os.path.getsize('tips_sample.csv')
gzip_size  = os.path.getsize('tips_sample.csv.gz')
parq_size  = os.path.getsize('tips_sample.parquet')

print(f"\nFile sizes — CSV: {csv_size} bytes | Gzipped CSV: {gzip_size} bytes | Parquet: {parq_size} bytes")


## 14) Join Patterns (10–15m)

In [ ]:
left = tips[['day','time','size']].drop_duplicates().copy()
right = tips[['day','time','tip']].groupby(['day','time']).mean().reset_index().rename(columns={'tip':'avg_tip'})
joined = left.merge(right, on=['day','time'], how='outer', indicator=True)
joined.head()

In [ ]:
# Right-only and inner (semi-join-style)
right_only = joined.loc[joined['_merge']=='right_only', right.columns]
inner_rows = joined.loc[joined['_merge']=='both', left.columns].drop_duplicates()
len(inner_rows)

In [ ]:
# merge_ordered and merge_asof demos
a = pd.DataFrame({'day':['Fri','Sat','Sun'], 'rank':[1,2,3]})
b = pd.DataFrame({'day':['Thur','Sat','Sun'], 'score':[70,80,90]})
ordered = pd.merge_ordered(a, b, on='day', how='outer')
events = pd.DataFrame({'when': pd.to_datetime(['2024-01-01 10:00','2024-01-02 12:30','2024-01-04 09:00']),'event':['A','B','C']}).sort_values('when')
measures = pd.DataFrame({'when': pd.to_datetime(['2024-01-01 09:45','2024-01-02 12:00','2024-01-03 18:00','2024-01-04 08:50']),'value':[10,20,15,30]}).sort_values('when')
asof_join = pd.merge_asof(events, measures, on='when', direction='nearest', tolerance=pd.Timedelta('1H'))
ordered.head(), asof_join.head()

### ✍️ Analysis (Joins) — 3–5 sentences
- Read `_indicator` results—what mismatches did you find?
- Where would `merge_asof` fit in a real pipeline?
- Any ordering assumptions for `merge_ordered`?

### 🔎 Quick checks
- Report counts for left/right/inner.
- Show 2 sample rows from an anti-join.

### ➕ Extension (pick one)
- Perform a semi-join and explain the use-case.
- Add a composite key and re-merge.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

left = tips[['day','time','size']].drop_duplicates().copy()
right = (
    tips[['day','time','tip']]
    .groupby(['day','time']).mean()
    .reset_index()
    .rename(columns={'tip':'avg_tip'})
)


joined = left.merge(right, on=['day','time'], how='outer', indicator=True)
print(joined.head())


right_only = joined.loc[joined['_merge']=='right_only', right.columns]
inner_rows = joined.loc[joined['_merge']=='both', left.columns].drop_duplicates()

print("\nCounts:")
print("Left:", (joined['_merge']=='left_only').sum())
print("Right:", (joined['_merge']=='right_only').sum())
print("Inner:", (joined['_merge']=='both').sum())


anti_join = joined.loc[joined['_merge']=='left_only', left.columns].head(2)
print("\nAnti-join sample:\n", anti_join)


a = pd.DataFrame({'day':['Fri','Sat','Sun'], 'rank':[1,2,3]})
b = pd.DataFrame({'day':['Thur','Sat','Sun'], 'score':[70,80,90]})
ordered = pd.merge_ordered(a, b, on='day', how='outer')
print("\nmerge_ordered:\n", ordered)


events = pd.DataFrame({
    'when': pd.to_datetime(['2024-01-01 10:00','2024-01-02 12:30','2024-01-04 09:00']),
    'event':['A','B','C']
}).sort_values('when')

measures = pd.DataFrame({
    'when': pd.to_datetime(['2024-01-01 09:45','2024-01-02 12:00','2024-01-03 18:00','2024-01-04 08:50']),
    'value':[10,20,15,30]
}).sort_values('when')

asof_join = pd.merge_asof(
    events, measures, on='when',
    direction='nearest',
    tolerance=pd.Timedelta('1H')
)

print("\nmerge_asof:\n", asof_join)


indicator shows that some (day,time) pairs exist in left_only (present in left but no tip avg in right) and some in right_only (aggregated tips without corresponding size row).
merge_asof is useful in time series pipelines—for example, attaching the closest sensor reading to an event timestamp within a tolerance.
merge_ordered assumes the join key is sorted (like dates or ranks) and keeps order intact; it’s best for chronological or ordinal joins rather than categorical ones.

In [ ]:
# Semi-join: keep only rows in left that have a match in right
semi_join = left[left[['day','time']].apply(tuple,1).isin(right[['day','time']].apply(tuple,1))]
print("\nSemi-join sample:\n", semi_join.head())

## 15) Method Chaining & `.pipe`, `.query`, `.eval` (10m)

In [ ]:
def add_bill_per_person(df): return df.assign(bill_per_person = df['total_bill']/df['size'])
pipe_demo = (tips.query("time=='Dinner'").pipe(add_bill_per_person)
             .groupby('day').agg(mean_bill_pp=('bill_per_person','mean'),
                                 mean_tip_pct=('tip_pct','mean'))
             .sort_values('mean_bill_pp', ascending=False))
eval_demo = tips.eval('bill_pp = total_bill / size')
pipe_demo.head(), eval_demo[['total_bill','size','bill_pp']].head()

### ✍️ Analysis (Method Chaining) — 3–5 sentences
- What made the chained pipeline clearer or riskier?
- Where would you break the chain for debugging?
- When is `.eval` useful vs risky?

### 🔎 Quick checks
- Show final columns of your pipeline result.
- Confirm idempotency by re-running.

### ➕ Extension (pick one)
- Refactor an earlier analysis into a chain; compare readability.
- Encapsulate a step into `.pipe` and reuse it.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

The chained pipeline is concise and highlights what is being done step-by-step without creating many intermediate variables, making it easier to read top-down.
However, long chains can be riskier to debug, since errors may propagate silently, and you lose visibility into intermediate states.
A good practice is to break the chain right after filtering or grouping when results are non-trivial.
.eval is useful for quick in-place column creation with less typing and sometimes performance gains, but can be risky when variable names shadow Python objects or when debugging errors.

In [ ]:

def add_bill_per_person(df): 
    return df.assign(bill_per_person = df['total_bill']/df['size'])

pipe_demo = (
    tips.query("time=='Dinner'")
        .pipe(add_bill_per_person)
        .groupby('day')
        .agg(mean_bill_pp=('bill_per_person','mean'),
             mean_tip_pct=('tip_pct','mean'))
        .sort_values('mean_bill_pp', ascending=False)
)


eval_demo = tips.eval('bill_pp = total_bill / size')

pipe_demo.head(), eval_demo[['total_bill','size','bill_pp']].head()


In [ ]:
#ouick checks
# # Show final columns of pipeline
pipe_demo.columns.tolist()

# Confirm idempotency (re-run and compare)
pipe_demo2 = (
    tips.query("time=='Dinner'")
        .pipe(add_bill_per_person)
        .groupby('day')
        .agg(mean_bill_pp=('bill_per_person','mean'),
             mean_tip_pct=('tip_pct','mean'))
        .sort_values('mean_bill_pp', ascending=False)
)
pipe_demo.equals(pipe_demo2)

#Extension
daily_chain = (
    tips_ts.resample('D')[['total_bill']].sum()
    .assign(roll14_med=lambda df: df['total_bill'].rolling(14, min_periods=1).median())
)
daily_chain.head()



## 16) Crosstab, `cut/qcut`, `where/mask` (5–10m)

In [ ]:
ct = pd.crosstab(tips['smoker'], tips['day'])
tips_bins = tips.copy()
tips_bins['bill_bucket'] = pd.cut(tips_bins['total_bill'], bins=[0,10,20,30,50])
bucket_mean = tips_bins.groupby('bill_bucket')['tip'].mean()
tips_q = tips.assign(q = pd.qcut(tips['tip_pct'], 4, duplicates='drop'))
ct, bucket_mean, tips_q.groupby('q')['total_bill'].mean()

### ✍️ Analysis (Crosstab / Bucketing) — 3–5 sentences
- What story does your crosstab tell in one sentence?
- How did `cut` vs `qcut` change bucket sizes?
- When is `where` preferable to boolean indexing?

### 🔎 Quick checks
- Show bucket edges and counts.
- Verify monotonicity of bucket means if expected.

### ➕ Extension (pick one)
- Swap `qcut`/`cut` and compare stats.
- Mask outliers and re-summarize.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

ct = pd.crosstab(tips['smoker'], tips['day'])


tips_bins = tips.copy()
tips_bins['bill_bucket'] = pd.cut(tips_bins['total_bill'], bins=[0,10,20,30,50])
bucket_mean = tips_bins.groupby('bill_bucket')['tip'].mean()


tips_q = tips.assign(q = pd.qcut(tips['tip_pct'], 4, duplicates='drop'))

ct, bucket_mean, tips_q.groupby('q')['total_bill'].mean()


The crosstab shows the distribution of smokers across days, e.g., more smokers typically appear on weekends.
cut produced equal-width bins, so some buckets had many rows (mid-range bills), while others were sparse (very high bills).
qcut instead forced equal-sized quantile groups, so each quartile has roughly the same number of rows but uneven numeric ranges.
where is preferable to boolean indexing when you want to preserve the original shape (same number of rows, with masked-out values replaced by NaN) instead of filtering rows out.

In [ ]:
#quick checks
# # Show bucket edges and counts
tips_bins['bill_bucket'].value_counts().sort_index()

# Verify monotonicity of bucket means 
bucket_mean

#Extension
tips_masked = tips.copy()
tips_masked['tip_masked'] = tips_masked['tip'].where(tips_masked['tip'] <= 9)


masked_mean = tips_masked.groupby('day')['tip_masked'].mean()
masked_mean


## 17) Styling & Export (5m)

In [ ]:
sty = (tips.groupby('day')[['total_bill','tip','tip_pct']].mean().round(2)
         .style.format({'tip_pct':'{:.2%}'}).background_gradient(axis=None))
html = sty.to_html()
open('tips_report.html','w',encoding='utf-8').write(html)
'Wrote tips_report.html'

### ✍️ Analysis (Styling & Export) — 3–5 sentences
- What formatting improved readability most?
- How would a stakeholder use this HTML report?
- One caution about styled HTML vs raw data files.

### 🔎 Quick checks
- Open the HTML to spot issues (NA/odd values).
- Confirm underlying numbers (pre-style) are correct.

### ➕ Extension (pick one)
- Add one more style and explain choice.
- Export both styled HTML and CSV for same table.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

sty = (
    tips.groupby('day')[['total_bill','tip','tip_pct']]
        .mean().round(2)
        .style
        .format({'tip_pct':'{:.2%}'})              
        .background_gradient(axis=None)            
)


html = sty.to_html()
open('tips_report.html','w',encoding='utf-8').write(html)
'Wrote tips_report.html'


The percentage formatting on tip_pct improved readability the most, since stakeholders can instantly see tipping rates without interpreting decimals.
The gradient background helps visually highlight higher or lower values without scanning every number closely.
A stakeholder would use this HTML report to quickly compare days for higher bills and tipping trends in a visually friendly format.
One caution: styled HTML is presentation-only; if someone needs raw numbers for further analysis, a CSV or Excel export is safer.

In [ ]:
#quick check
tips.groupby('day')[['total_bill','tip','tip_pct']].mean().round(2)

#Extension
# Save raw data as CSV
raw_tbl = tips.groupby('day')[['total_bill','tip','tip_pct']].mean().round(2)
raw_tbl.to_csv('tips_report.csv')

'Wrote tips_report.html and tips_report.csv'


# Capstone Mini-Project (Free Dataset): **Titanic Survival Analysis** (30–45m)

**Source:** https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

In [ ]:
tit_url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv"
titanic = pd.read_csv(tit_url)
titanic.head()

In [ ]:
print("Shape:", titanic.shape)
print("\nInfo:"); print(titanic.info())
print("\nMissing values per column:"); print(titanic.isna().sum().sort_values(ascending=False).head(12))

## A) Data Cleaning & Feature Engineering

In [ ]:
df = titanic.copy()
for col in ['sex','class','embarked','embark_town','alive','who','deck','alone','adult_male']:
    if col in df.columns: df[col] = df[col].astype('category')
age_med = df.groupby(['sex','class'])['age'].transform('median')
df['age'] = df['age'].fillna(age_med)
if df['embark_town'].isna().any():
    mode_town = df['embark_town'].mode(dropna=True)
    if not mode_town.empty: df['embark_town'] = df['embark_town'].fillna(mode_town.iloc[0])
df['family_size'] = df['sibsp'].fillna(0) + df['parch'].fillna(0) + 1
df['fare_pp'] = df['fare'] / df['family_size']
df['deck'] = df['deck'].cat.add_categories(['Unknown']).fillna('Unknown')
bins = [0,12,18,35,50,80]; labels = ['Child','Teen','YoungAdult','MidAge','Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, include_lowest=True)
df.head()

### ✅ Exercise A
Check NaNs in `age`/`embark_town`, top `family_size`, % with `fare_pp>20`.

**Hints:**
- Concept: impute & feature engineer.
- API: `groupby().transform('median')`, `.mode()`, `.cut`.


In [ ]:
# YOUR WORK: Exercise A

df[['age','embark_town']].isna().sum()

df['family_size'].value_counts().head()

pct_high_fare = (df['fare_pp'] > 20).mean() * 100
pct_high_fare


## B) Survival Patterns

In [ ]:
overall_survival = df['survived'].mean(); print('Overall survival rate:', round(overall_survival,3))

In [ ]:
surv_by_class = df.groupby('class')['survived'].mean().reindex(['First','Second','Third'])
plt.figure(); plt.bar(surv_by_class.index.astype(str), surv_by_class.values)
plt.title('Survival Rate by Class'); plt.xlabel('class'); plt.ylabel('survival rate'); plt.show()

In [ ]:
piv = df.pivot_table(index='sex', columns='class', values='survived', aggfunc='mean')
plt.figure(); plt.imshow(piv.values, aspect='auto')
plt.title('Survival Rate by Sex × Class'); plt.xlabel('class'); plt.ylabel('sex')
plt.xticks(range(len(piv.columns)), piv.columns.astype(str))
plt.yticks(range(len(piv.index)), piv.index.astype(str))
plt.colorbar(); plt.show(); piv

### ✅ Exercise B
1) Histogram of age. 2) Boxplot fare by class. 3) Bar of survival by age_group.

**Hints:**
- Concept: univariate + grouped visuals.
- API: `plt.hist`, `plt.boxplot`, `groupby().mean` + `plt.bar`.


In [ ]:
# YOUR WORK: Exercise B
import matplotlib.pyplot as plt


plt.figure(figsize=(6,4))
plt.hist(df['age'].dropna(), bins=30, color='skyblue', edgecolor='black')
plt.title("Distribution of Passenger Age")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()


plt.figure(figsize=(6,4))
plt.boxplot([df.loc[df['class']=='First','fare'],
             df.loc[df['class']=='Second','fare'],
             df.loc[df['class']=='Third','fare']],
            labels=['First','Second','Third'])
plt.title("Fare by Passenger Class")
plt.xlabel("Class")
plt.ylabel("Fare")
plt.show()

surv_by_agegrp = df.groupby('age_group')['survived'].mean()
plt.figure(figsize=(6,4))
plt.bar(surv_by_agegrp.index.astype(str), surv_by_agegrp.values, color='lightgreen', edgecolor='black')
plt.title("Survival Rate by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Survival Rate")
plt.show()

surv_by_agegrp


## C) Subgroup Discovery

In [ ]:
grp = (df.groupby(['sex','class','embark_town'])
         .agg(n=('survived','size'), surv_rate=('survived','mean'))
         .query('n >= 25')
         .sort_values('surv_rate', ascending=False))
grp.head(10)

### ✅ Exercise C
Families with `family_size≥4`: survival by `sex` & `class` (n + rate). Bucket `fare_pp` and compute survival per bucket.

**Hints:**
- Concept: subgroup analysis with thresholds.
- API: `groupby().agg`, `query`, `cut`.


In [ ]:
# YOUR WORK: Exercise C

fam4 = (df.query("family_size >= 4")
          .groupby(['sex','class'])
          .agg(n=('survived','size'),
               surv_rate=('survived','mean'))
          .sort_values('surv_rate', ascending=False))
print("Survival by Sex × Class (Families ≥4):")
print(fam4)


fare_bins = [0,5,10,20,50,100,600]
fare_labels = ['≤5','5–10','10–20','20–50','50–100','100+']
df['fare_pp_bucket'] = pd.cut(df['fare_pp'], bins=fare_bins, labels=fare_labels, include_lowest=True)

fare_bucket_surv = (df.groupby('fare_pp_bucket')
                      .agg(n=('survived','size'),
                           surv_rate=('survived','mean'))
                      .sort_index())

print("\nSurvival by Fare per Person Bucket:")
print(fare_bucket_surv)


## D) Deliverables

In [ ]:
top10 = grp.head(10).reset_index()
sty = (top10.style.format({'surv_rate':'{:.2%}'}).hide(axis='index'))
html = sty.to_html()
open('titanic_top10_survival.html','w',encoding='utf-8').write(html)
top10.to_csv('titanic_top10_survival.csv', index=False)
"Saved titanic_top10_survival.html and titanic_top10_survival.csv"

### ✍️ Analysis (Capstone — Titanic) — 3–5 sentences
- State your main finding (who/when/context) with one number and one plot reference.
- Name one limitation in your approach (bias, leakage, imputation).
- Suggest a next step or feature to add.

### 🔎 Quick checks
- Link the exact cell/table backing your claim.
- Confirm deliverables (HTML + CSV) were written.

### ➕ Extension (pick one)
- Add an extra feature (e.g., `is_child`, z-scored fare) and see if subgroup ranking changes.
- Try a different minimum group size and discuss stability.

**Rubric (10 pts):** Correctness 3 • Evidence 2 • Clarity 2 • Reproducibility 2 • Exploration 1

In [ ]:

df['is_child'] = (df['age'] < 12).astype(int)


grp_child = (df.groupby(['is_child','sex','class','embark_town'])
               .agg(n=('survived','size'),
                    surv_rate=('survived','mean'))
               .query('n >= 25')
               .sort_values('surv_rate', ascending=False))

top10_child = grp_child.head(10).reset_index()
print(top10_child)


top10_child.to_csv("titanic_top10_survival_with_child.csv", index=False)


In [ ]:

import os

print("HTML exists?", os.path.exists("titanic_top10_survival.html"))
print("CSV exists?", os.path.exists("titanic_top10_survival.csv"))


check_df = pd.read_csv("titanic_top10_survival.csv")
print("Shape:", check_df.shape)
print(check_df.head())


Our main finding is that female passengers in First Class embarking from Cherbourg had the highest survival rate (≈97%), as shown in the Top-10 survival subgroup table (see D, grp.head(10)) and supported by the pivot heatmap in Section B. This highlights how wealth and embarkation port intersected with gender to shape survival outcomes.
A limitation of this approach is that we rely on group averages with simple thresholds (n ≥ 25), which may mask variability and also bias results if missing data (e.g., age, embarkation town) were imputed or dropped unevenly.
As a next step, we could introduce a derived feature like is_child (age < 12) or normalize fare (z-score) to see whether children or unusually expensive tickets within the same class change subgroup rankings. This would help refine the notion of privilege and vulnerability beyond just class/sex categories.
